In [1]:
import sys
sys.path.insert(1, '/Users/mac/Desktop/PycharmProjects/TAADL/src')
sys.path.insert(2, '/Users/mac/Desktop/PycharmProjects/TAADL/models')

import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import json

from torch import optim
from network import GPCopulaRNN
from config import DATA_PATH, E_DICT
from utils import transform, train_test_split, load_data

In [11]:
# load and preprocess data
prices_m = load_data('M')

l_ret = np.log(prices_m/prices_m.shift()).fillna(0.0)
z_tr, z_te = train_test_split(prices_m) 
x_tr, cdf  = transform(z_tr)

In [17]:
from scipy.interpolate import interp1d

# TODO: current CDF does not map the x to y correctly. output is just monotone increasing.
# map the states accrodingly.

class ECDF_(object):
    def __init__(self, data:np.ndarray):
        self.x = data 
        self.x_min, self.x_max = self.x.min(), self.x.max()
        self.n = len(self.x)
        self.y = np.linspace(1.0/self.n, 1.0, self.n)
        self.f = interp1d(self.x, self.y, fill_value='extrapolate') # make interpolation
        self.inv_f = interp1d(self.y, self.x, fill_value='extrapolate') # inverse is just arguments reversed
        
    def __call__(self, x_:np.ndarray) -> np.ndarray:
        """
        calculates y given x under defined ECDF class.
        """
        if np.sum(x_ > self.x_max) > 0 or np.sum(x_ < self.x_min) > 0:
            x_ = np.where(x_ > self.x_max, self.x_max, x_)
            x_ = np.where(x_ < self.x_min, self.x_min, x_)
            print(x_)
        return self.f(x_)

    def inverse(self, y:np.ndarray) -> np.ndarray:
        """
        calculates the inverse of ECDF with y as an input.
        """
         # if cdf value is less than 0 or more than 1, trim values
        if np.sum(y > 1.0) > 0 or np.sum(y < 0.0) > 0:
            y = np.where(y > 1.0, 1.0, y)
            y = np.where(y < 0.0, 0.0, y)
        return self.inv_f(y) # otherwise, return

In [14]:
cdf = ECDF_(z_tr[:,0].numpy())

In [18]:
z_tr[:,0].numpy()

array([ 0.        ,  0.04308827, -0.02488683, -0.01671686, -0.00568048,
        0.01038912,  0.03261695,  0.02982194,  0.01564011, -0.03088782,
        0.01524771,  0.04128616,  0.05644295,  0.00198712,  0.03691364,
        0.05079815, -0.03019079,  0.00714155,  0.01951302,  0.0296161 ,
        0.00664789,  0.03818868,  0.03524607,  0.02959223,  0.01136696,
       -0.01298764,  0.03322056,  0.0587876 ,  0.02152496, -0.00401167,
       -0.0261666 , -0.00484038,  0.05023453,  0.04430554, -0.0338739 ,
       -0.02488002, -0.0920969 , -0.00470977,  0.01168569,  0.04332916,
        0.00409524, -0.09817712, -0.02605131, -0.04050545, -0.13189924,
       -0.23888402, -0.06958473,  0.07415045, -0.15946335, -0.11271114,
        0.07549214,  0.11995257,  0.12561159, -0.01895484,  0.10373427,
        0.05259912,  0.0491927 , -0.02309832,  0.04338939, -0.00184333,
       -0.06824145, -0.01021957,  0.06241927, -0.04018088, -0.12077456,
       -0.01906217,  0.12937592, -0.04852358,  0.1075642 ,  0.04

In [19]:
x = cdf(z_tr[:,0].numpy())
x

array([0.00694444, 0.01388889, 0.02083333, 0.02777778, 0.03472222,
       0.04166667, 0.04861111, 0.05555556, 0.0625    , 0.06944444,
       0.07638889, 0.08333333, 0.09027778, 0.09722222, 0.10416667,
       0.11111111, 0.11805556, 0.125     , 0.13194444, 0.13888889,
       0.14583333, 0.15277778, 0.15972222, 0.16666667, 0.17361111,
       0.18055556, 0.1875    , 0.19444444, 0.20138889, 0.20833333,
       0.21527778, 0.22222222, 0.22916667, 0.23611111, 0.24305556,
       0.25      , 0.25694444, 0.26388889, 0.27083333, 0.27777778,
       0.28472222, 0.29166667, 0.29861111, 0.30555556, 0.3125    ,
       0.31944444, 0.32638889, 0.33333333, 0.34027778, 0.34722222,
       0.35416667, 0.36111111, 0.36805556, 0.375     , 0.38194444,
       0.38888889, 0.39583333, 0.40277778, 0.40972222, 0.41666667,
       0.42361111, 0.43055556, 0.4375    , 0.44444444, 0.45138889,
       0.45833333, 0.46527778, 0.47222222, 0.47916667, 0.48611111,
       0.49305556, 0.5       , 0.50694444, 0.51388889, 0.52083

In [2]:
#TODO add mean and covariance to existing RP strategy and do the plotting
params = torch.load('/Users/mac/Desktop/PycharmProjects/TAADL/models/GaussCopula.pt')

In [23]:
num_samples = 20
num_epochs = 250

# load parameters for model and optimizer
model = GPCopulaRNN(input_size=1, hidden_size=4, num_layers=2, rank_size=5, 
                        batch_size=3, num_asset=7, dropout=0.05, features=e_dict)
model.load_state_dict(params['net_params'])

<All keys matched successfully>

In [69]:
num_layers = 2
num_assets = 7
hidden_size = 4
seq_length = 12
batch_size = 1

lstm = nn.LSTM(input_size=1, hidden_size=hidden_size,
                    num_layers=num_layers, batch_first=False) # num_batch x seq_len x num_feature


h0, c0 = torch.zeros(num_layers, batch_size, hidden_size), torch.zeros(num_layers, batch_size, hidden_size)
test = torch.rand(12,1,7)

outputs = []
for i in range(test.size(2)):
    output, (hn, cn) = lstm(test[:,:,i].unsqueeze(2), (h0,c0))
    outputs.append(output)

In [71]:
outputs[0]

tensor([[[-0.0681,  0.0067, -0.0214, -0.0057]],

        [[-0.0961,  0.0206, -0.0354, -0.0099]],

        [[-0.1068,  0.0381, -0.0508, -0.0132]],

        [[-0.1140,  0.0494, -0.0518, -0.0170]],

        [[-0.1176,  0.0581, -0.0528, -0.0192]],

        [[-0.1196,  0.0651, -0.0544, -0.0205]],

        [[-0.1218,  0.0669, -0.0496, -0.0210]],

        [[-0.1227,  0.0690, -0.0489, -0.0205]],

        [[-0.1237,  0.0689, -0.0462, -0.0197]],

        [[-0.1240,  0.0708, -0.0481, -0.0191]],

        [[-0.1241,  0.0742, -0.0524, -0.0195]],

        [[-0.1249,  0.0749, -0.0512, -0.0210]]], grad_fn=<StackBackward0>)

In [62]:
test2 = test.permute(0,2,1)
h0_2, c0_2 = torch.zeros(num_layers, seq_length, hidden_size), torch.zeros(num_layers, seq_length, hidden_size)

In [63]:
lstm2 = nn.LSTM(input_size=1, hidden_size=hidden_size,
                num_layers=num_layers, batch_first=True)

output2, (hn_2, cn_2) = lstm2(test2, (h0_2,c0_2))


In [68]:
output2.size()

torch.Size([12, 7, 4])

In [76]:
 outputs[0]

tensor([[[-0.0681,  0.0067, -0.0214, -0.0057]],

        [[-0.0961,  0.0206, -0.0354, -0.0099]],

        [[-0.1068,  0.0381, -0.0508, -0.0132]],

        [[-0.1140,  0.0494, -0.0518, -0.0170]],

        [[-0.1176,  0.0581, -0.0528, -0.0192]],

        [[-0.1196,  0.0651, -0.0544, -0.0205]],

        [[-0.1218,  0.0669, -0.0496, -0.0210]],

        [[-0.1227,  0.0690, -0.0489, -0.0205]],

        [[-0.1237,  0.0689, -0.0462, -0.0197]],

        [[-0.1240,  0.0708, -0.0481, -0.0191]],

        [[-0.1241,  0.0742, -0.0524, -0.0195]],

        [[-0.1249,  0.0749, -0.0512, -0.0210]]], grad_fn=<StackBackward0>)

In [78]:
output2[:,0,:]

tensor([[-0.0227, -0.1578,  0.0350, -0.0186],
        [-0.0089, -0.1551,  0.0307, -0.0177],
        [-0.0150, -0.1564,  0.0326, -0.0181],
        [-0.0154, -0.1564,  0.0327, -0.0182],
        [-0.0251, -0.1583,  0.0357, -0.0188],
        [-0.0039, -0.1540,  0.0290, -0.0174],
        [-0.0107, -0.1555,  0.0312, -0.0179],
        [-0.0307, -0.1592,  0.0373, -0.0191],
        [-0.0148, -0.1563,  0.0325, -0.0181],
        [-0.0087, -0.1551,  0.0306, -0.0177],
        [-0.0041, -0.1540,  0.0290, -0.0174],
        [-0.0070, -0.1547,  0.0300, -0.0176]], grad_fn=<SliceBackward0>)

In [79]:
outputs[1]

tensor([[[-0.0684,  0.0060, -0.0201, -0.0054]],

        [[-0.0970,  0.0173, -0.0299, -0.0087]],

        [[-0.1083,  0.0314, -0.0402, -0.0111]],

        [[-0.1133,  0.0456, -0.0501, -0.0136]],

        [[-0.1173,  0.0551, -0.0515, -0.0166]],

        [[-0.1191,  0.0639, -0.0555, -0.0188]],

        [[-0.1207,  0.0702, -0.0572, -0.0210]],

        [[-0.1208,  0.0779, -0.0639, -0.0225]],

        [[-0.1227,  0.0797, -0.0600, -0.0251]],

        [[-0.1231,  0.0828, -0.0613, -0.0262]],

        [[-0.1230,  0.0868, -0.0656, -0.0271]],

        [[-0.1240,  0.0876, -0.0636, -0.0290]]], grad_fn=<StackBackward0>)

In [80]:
output2[:,1,:]

tensor([[-0.0071, -0.2588,  0.0495, -0.0176],
        [ 0.0045, -0.2580,  0.0460, -0.0176],
        [ 0.0022, -0.2579,  0.0467, -0.0174],
        [ 0.0033, -0.2577,  0.0463, -0.0173],
        [-0.0081, -0.2588,  0.0497, -0.0175],
        [ 0.0308, -0.2528,  0.0364, -0.0158],
        [ 0.0159, -0.2556,  0.0419, -0.0166],
        [ 0.0029, -0.2556,  0.0453, -0.0162],
        [ 0.0171, -0.2547,  0.0412, -0.0162],
        [ 0.0173, -0.2555,  0.0415, -0.0166],
        [ 0.0136, -0.2567,  0.0429, -0.0172],
        [ 0.0261, -0.2536,  0.0381, -0.0160]], grad_fn=<SliceBackward0>)